In [1]:
import os
from Bio import Entrez
import xml.etree.ElementTree as ET
import subprocess
import pandas

Entrez.email = 'mfoster11@mgh.harvard.edu'

In [2]:
metadata_v8 = '../notebooks/metadata/cleaned_metadata/longread_metadata_v8.csv'
sra_dir = 'fastq'
assemblies_dir = '../longread_analysis/paired_assemblies/paired_only/longread'
acc_list = 'SraAccList.csv'
edirect = '/home/mf019/edirect'
prefetch = '/home/mf019/sratoolkit/bin/prefetch'

In [3]:
v8_df = pandas.read_csv(metadata_v8)
name_map = v8_df[['Isolate', 'Original_Name']]
old2new = {}
for idx, row in name_map.iterrows():
    new_name = row['Isolate']
    old_name = row['Original_Name']
    print(new_name, old_name)
    old2new[old_name] = new_name
    

ESI26 B1904/13
ESI26H EU57_B190413
UCT109 K69
UCT109H K69
UCT110 K70
UCT110H K70
UCT113 K76
UCT113H K76
UCT29 K1
UCT29H K1
UCT30 K11
UCT30H K11
UCT31 K10
UCT31H K10
UCT32 K15
UCT32H K15
UCT35 K24
UCT35H K24
UCT50 K21
UCT50H K21
UCT92 K72
UCT92H K72
UCT96 K84
UCT96H K84
UNY1032 BL206
UNY1032P BL206
UNY1038 BL224
UNY1038P BL224
UNY1083 BL515
UNY1083P BL515
UNY1085 BL522
UNY1085P BL522
UNY1090 BL546
UNY1090P BL546
UNY1128 MR614
UNY1128P MR614
UNY149 B247
UNY149P B247
UNY169 B348
UNY169P B348
UNY172 B356
UNY172P B356
UNY193 B477
UNY193P B477
UNY203 MR616
UNY203P MR616
UNY208 MR641
UNY208P MR641
UNY990 B483
UNY990P B483
URI101 K57
URI101H K57
URI102 K58
URI102H K58
URI103 K59
URI103H K59
URI107 K66
URI107H K66
URI111 K74
URI111H K74
URI112 K75
URI112H K75
URI117 K82
URI117H K82
URI118 K83
URI118H K83
URI120 K86
URI120H K86
URI33 K16
URI33H K16
URI34 K22
URI34H K22
URI36 K25
URI36H K25
URI39 K2
URI39H K2
URI40 K3
URI40H K3
URI41 K6
URI41H K6
URI42 K7
URI42H K7
URI44 K12
URI44H K12
URI46 K17


In [4]:
def print_xml_structure(element, level=0):
    """Helper function to print XML structure"""
    indent = '  ' * level
    print(f"{indent}Tag: {element.tag}")
    if element.attrib:
        print(f"{indent}Attributes: {element.attrib}")
    if element.text and element.text.strip():
        print(f"{indent}Text: {element.text.strip()}")
    for child in element:
        print_xml_structure(child, level + 1)

In [26]:
samples = {}
sra_to_sid = {}
with open(acc_list, 'r') as infile:
    lines = infile.readlines()
    for line in lines:
        sra_id = line.strip()
        handle = Entrez.efetch(db='sra', id=sra_id)
        record = handle.read().decode('utf-8')
        handle.close()
        
        root = ET.fromstring(record)
        sample = root.find(".//SAMPLE")
        sample_name = sample.get('alias')
        sra_to_sid[sra_id] = sample_name
        new_name = old2new.get(sample_name, '')
        print(sra_id, sample_name, new_name)
        if new_name != '':
            outdir = f'fastq/{new_name}'
        else:
            outdir = f'fastq/{new_name}'
        os.makedirs(outdir, exist_ok=True)
        fetch_cmd = f"bash -c 'source ~/.bashrc' && {prefetch} {sra_id} -O {outdir}"
        subprocess.run(fetch_cmd, shell=True)

SRR9616101 K57 URI101H
2024-10-28T21:04:16 prefetch.3.1.1: 1) Resolving 'SRR9616101'...
2024-10-28T21:04:16 prefetch.3.1.1: Current preference is set to retrieve SRA Normalized Format files with full base quality scores
2024-10-28T21:04:17 prefetch.3.1.1: 1) Downloading 'SRR9616101'...
2024-10-28T21:04:17 prefetch.3.1.1:  SRA Normalized Format file is being retrieved
2024-10-28T21:04:17 prefetch.3.1.1:  Downloading via HTTPS...
2024-10-28T21:04:17 prefetch.3.1.1:    Continue download of 'SRR9616101' from 364993081
2024-10-28T21:08:26 prefetch.3.1.1:  HTTPS download succeed
2024-10-28T21:08:27 prefetch.3.1.1:  'SRR9616101' is valid: 685032002 bytes were resumed from 364993081
2024-10-28T21:08:27 prefetch.3.1.1: 1) 'SRR9616101' was downloaded successfully
2024-10-28T21:08:27 prefetch.3.1.1: 'SRR9616101' has 0 unresolved dependencies
SRR9616102 K66 URI107H
2024-10-28T21:08:28 prefetch.3.1.1: 1) Resolving 'SRR9616102'...
2024-10-28T21:08:29 prefetch.3.1.1: Current preference is set to retr

HTTPError: HTTP Error 400: Bad Request

In [15]:
import glob
glob.glob(f'{sra_dir}/*/*/*.sra')

['fastq/K66/SRR9616102/SRR9616102.sra',
 'fastq/K83/SRR9616106/SRR9616106.sra',
 'fastq/K82/SRR9616105/SRR9616105.sra',
 'fastq/K76/SRR9616104/SRR9616104.sra',
 'fastq/K84/SRR9616107/SRR9616107.sra',
 'fastq/K57/SRR9616101/SRR9616101.sra',
 'fastq/K75/SRR9616103/SRR9616103.sra']